<a href="https://colab.research.google.com/github/mrab72/user-behave-prediction/blob/master/SVM_classifier_categories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Connect to the google drive:



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Import necessary packages

In [ ]:
import json
from collections import defaultdict
import operator
from django.utils.dateparse import parse_datetime
import seaborn as sns
import pandas as pd
import numpy as np
from __future__ import absolute_import, division, print_function, unicode_literals

from matplotlib import pyplot as plt
from datetime import datetime, timedelta
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn import svm
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
tf.compat.v1.enable_eager_execution

import numpy as np
import os
import time
import random
from collections import OrderedDict

In [ ]:
from sklearn import svm
clf = svm.SVC()
clf.fit(feature_arr, y)

## Load 84:241:9:79 Ip Data:

In [ ]:
with open ('/content/drive/My Drive/tubp/file_84_241_9_79_apr.json', 'r') as f:
  data = json.load(f)

In [ ]:
with open ('/content/drive/My Drive/tubp/file_84_241_9_79_may.json', 'r') as f:
  data_test = json.load(f)

## Split Data into the n_minutes chunks:

### parsed date from data:


In [ ]:
def _parsed_data(data):
  parsed_data = {}
  for date, traffic in data.items():
    parsed_date = datetime.strptime(date.split('.')[0], '%Y-%m-%d-%H:%M:%S')
    parsed_data[parsed_date] = traffic
  return OrderedDict(sorted(parsed_data.items(), key=lambda t: t[0]))

In [ ]:
ordered_data = _parsed_data(data)
ordered_data_test = _parsed_data(data_test)

In [ ]:
def _internet_seq_chunk(ordered_data):
  internet_seq_chunk = defaultdict(list)
  for date, traffic_type in ordered_data.items():
    new_date = (date - timedelta(0, 60 * (date.minute  % 60))).replace(second=0)
    
    internet_seq_chunk[new_date].append(traffic_type)
  return internet_seq_chunk
internet_seq_chunk = _internet_seq_chunk(ordered_data)
internet_seq_chunk_test = _internet_seq_chunk(ordered_data_test)


In [ ]:
def create_type_seq(category, internet_seq_chunk):
  cnt = 0
  internet_seq = defaultdict()
  for date, traffic_list in internet_seq_chunk.items():
    cnt += len(traffic_list)
    if category in traffic_list:
      internet_seq[date] = 1
    else:
      internet_seq[date] = 0

  internet_seq = OrderedDict(sorted(internet_seq.items(), key=lambda t: t[0]))
  return internet_seq, cnt


In [ ]:
res, cnt = create_type_seq("Web", internet_seq_chunk)

In [ ]:
distribution = {}
for category in types:
  res = create_type_seq(category, internet_seq_chunk)
  distribution[category] = sum(list(res.values()))

In [ ]:
len(internet_seq_chunk)

In [ ]:
total_seq = OrderedDict(([(datetime(2020, 1, 6, 12, 55), 0)]))
date = datetime(2020, 1, 6, 12, 55)

while date <= datetime(2020,4,14,7,10):
  date = date + timedelta(0, 300)
  total_seq[date] = sequence.get(date, 0)

In [ ]:
total_seq_test = OrderedDict(([(datetime(2020, 4, 14, 7, 10), 0)]))
date = datetime(2020, 4, 14, 7, 10)

while date <= datetime(2020, 4, 29, 6, 10):
  date = date + timedelta(0, 300)
  total_seq_test[date] = sequence_test.get(date, 0)
  


### Prepare dataset :


*   one hot vector for day of week feature
*   one hot vector for hour feature
*   60 previous step as  feature



In [ ]:
def prepare_x_y(seq):
  df = pd.DataFrame.from_dict(seq, orient='index').reset_index()
  df = df.rename(columns={"index": "time", 0: "usage"})

  day_of_weeks = pd.get_dummies(list(map( lambda x: x.dayofweek, list(df.time)))).values
  hour = pd.get_dummies(list(map(lambda x: x.hour, list(df.time)))).values
  minutes = pd.get_dummies(list(map(lambda x: x.minute, list(df.time)))).values

  previous_steps = []
  for i in range(len(df.usage.values)):
    if i >= 20:
      previous_steps.append(list(df.usage)[i - 20:i])
    else:
      previous_steps.append([0 for i in range(20)])

  previous_steps_df = pd.DataFrame(previous_steps).values

  return np.hstack((minutes, hour, day_of_weeks, previous_steps_df)), df.usage.values

In [ ]:
x_test, y_test  = prepare_x_y(total_seq_test)

In [ ]:
y_predict = clf.predict(x_test)

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier

In [ ]:
precision_score(y_test, y_predict), recall_score(y_test, y_predict), accuracy_score(y_test, y_predict), f1_score(y_test, y_predict)

# Define a SVMClassifier class object 

In [ ]:
class CategorySVMClassfier:

  def __init__(self, category, train_file, test_file, n_minutes=5):
    self.category = category 
    self.data = self._read_data_file(train_file)
    self.data_test = self._read_data_file(test_file)
    self.model = DecisionTreeClassifier(random_state=0) # svm.SVC()
    self.n_minutes = n_minutes

  def _read_data_file(self, directory):
    with open(directory) as f:
      return json.load(f)

  def _parsed_data(self, data):
    parsed_data = {}
    for date, traffic in data.items():
      parsed_date = datetime.strptime(date.split('.')[0], '%Y-%m-%d-%H:%M:%S')
      parsed_data[parsed_date] = traffic
    return OrderedDict(sorted(parsed_data.items(), key=lambda t: t[0]))

  def _internet_seq_chunk(self, ordered_data):
    internet_seq_chunk = defaultdict(list)
    for date, traffic_type in ordered_data.items():
      new_date = (date - timedelta(0, 60 * (date.minute  % self.n_minutes))).replace(second=0)
      internet_seq_chunk[new_date].append(traffic_type)
      
    return internet_seq_chunk

  def create_type_seq(self, category, internet_seq_chunk):
    internet_seq = defaultdict()
    for date, traffic_list in internet_seq_chunk.items():
      if category in traffic_list:
        internet_seq[date] = 1
      else:
        internet_seq[date] = 0

    internet_seq = OrderedDict(sorted(internet_seq.items(), key=lambda t: t[0]))
    return internet_seq

  def _prepare_x_y(self, seq, usage):
    df = pd.DataFrame.from_dict(seq, orient='index').reset_index()
    df = df.rename(columns={"index": "time", 0: "usage"})

    internt_usage_df = pd.DataFrame.from_dict(usage, orient='index').reset_index()
    internt_usage_df = internt_usage_df.rename(columns={"index": "time", 0: "usage"})

    day_of_weeks = pd.get_dummies(list(map( lambda x: x.dayofweek, list(df.time)))).values
    hour = pd.get_dummies(list(map(lambda x: x.hour, list(df.time)))).values
    minutes = pd.get_dummies(list(map(lambda x: x.minute, list(df.time)))).values

    previous_steps = []
    previous_usage = []
    for i in range(len(df.usage.values)):
      if i >= 100:
        previous_steps.append(list(df.usage)[i - 100:i])
        previous_usage.append(list(internt_usage_df.usage)[i - 100:i])
        
      else:
        previous_steps.append([0 for i in range(100)])
        previous_usage.append([0 for i in range(100)])

    previous_steps_df = pd.DataFrame(previous_steps).values
    previous_usage_df = pd.DataFrame(previous_usage).values

    return np.hstack((minutes, hour, day_of_weeks, previous_steps_df)), df.usage.values

  def _cover_all_times(self, seq):
    total_seq = OrderedDict(([(min(list(seq.keys())), 0)]))
    date = min(list(seq.keys()))
    internet_usage = OrderedDict(([(min(list(seq.keys())), 0)]))
    while date <= max(list(seq.keys())):
      date = date + timedelta(0, 300)
      total_seq[date] = seq.get(date, 0)
      res = seq.get(date, None)
      internet_usage[date]= 1 if res is not None else 0
    return total_seq, internet_usage

  def _prepare_data(self, data):
    ordered_data = self._parsed_data(data)
    internet_seq_chunk = self._internet_seq_chunk(ordered_data)
    sequence = self.create_type_seq(self.category, internet_seq_chunk)
    return self._cover_all_times(sequence)

  def train(self):
    total_seq, usage = self._prepare_data(self.data)
    self.predict_len = len(total_seq)
    print("start creating features array")
    x, y = self._prepare_x_y(total_seq, usage)
    print("**************")
    print("start training model")
    self.model.fit(x, y)
    print("*********************")
  
  def evaluate(self):
    total_seq, internet_usage = self._prepare_data(self.data_test)
    x_test, y_test = self._prepare_x_y(total_seq, internet_usage)
    y_predict = self.model.predict(x_test[self.predict_len:])

    return {
        "precision": precision_score(y_test[self.predict_len:], y_predict),
        "recall": recall_score(y_test[self.predict_len:], y_predict),
        "accuaracy": accuracy_score(y_test[self.predict_len:], y_predict),
        "f1_score": f1_score(y_test[self.predict_len:], y_predict)
    }

# train and evaluate model per category type:


In [ ]:
types = [
'Chat',
'Cloud',
'Database',
'Email',
'RemoteAccess',
'SocialNetwork',
'System',
'Unspecified',
'VoIP',
'VPN',
'Web'
]

In [ ]:
for category in types:
  print("$$$$$$$$$$$%s$$$$$$$$$$$$"%category)
  for step in [5, 15, 30, 60]:
    
    train_file = '/content/drive/My Drive/tubp/file_84_241_9_79_apr.json'
    test_file = '/content/drive/My Drive/tubp/file_84_241_9_79_may.json'
    clf = CategorySVMClassfier(category, train_file, test_file, n_minutes=step)
    clf.train()
    print("***********%s***********" % step)
    print("+++++++++++++++result++++++++++++++")
    print(clf.evaluate())
    
  print("^^^^^^^^^^^^^^^^^^^END OF %s^^^^^^^^^^^^^^^^^^" % category)